In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Generate synthetic sequential data
n_samples = 100
n_features = 3
sequence_length = 10

# Generate random features
features = np.random.rand(n_samples, sequence_length, n_features)
labels = np.random.randint(1, 2, size=n_samples)

In [3]:
# Split data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(features, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [4]:
# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, n_features)).reshape(-1, sequence_length, n_features)
X_val_scaled = scaler.transform(X_val.reshape(-1, n_features)).reshape(-1, sequence_length, n_features)
X_test_scaled = scaler.transform(X_test.reshape(-1, n_features)).reshape(-1, sequence_length, n_features)

In [5]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [6]:
# Define LSTM model
# class LSTMModel(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(LSTMModel, self).__init__()
#         self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
#         self.fc = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         out, _ = self.lstm(x)
#         out = self.fc(out[:, -1, :])
#         return out


class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)  # Additional LSTM layer
        self.fc1 = nn.Linear(hidden_size, 64)  # First dense layer
        self.fc2 = nn.Linear(64, 32)  # Second dense layer
        self.fc3 = nn.Linear(32, output_size)  # Third dense layer

    def forward(self, x):
        out, _ = self.lstm1(x)
        out, _ = self.lstm2(out)  # Pass through the second LSTM layer
        out = self.fc1(out[:, -1, :])
        out = self.fc2(out)
        out = self.fc3(out)
        return out


In [7]:
# Initialize model
# input_size = n_features
# hidden_size = 64
# output_size = 1
# model = LSTMModel(input_size, hidden_size, output_size)

# # Define loss function and optimizer
# criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# Instantiate the enhanced model
input_size = n_features
hidden_size = 64
output_size = 1
enhanced_model = LSTMModel(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(enhanced_model.parameters(), lr=0.001)

In [10]:
# Training loop
n_epochs = 10
# for epoch in range(n_epochs):
#     model.train()
#     optimizer.zero_grad()
#     outputs = model(X_train_tensor)
#     loss = criterion(outputs, y_train_tensor.unsqueeze(1))
#     loss.backward()
#     optimizer.step()
#     print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}")

# Training loop (adjust as needed)
for epoch in range(n_epochs):
    enhanced_model.train()
    optimizer.zero_grad()
    outputs = enhanced_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.unsqueeze(1))
    loss.backward()
    optimizer.step()

    # Validation
    enhanced_model.eval()
    with torch.no_grad():
        val_outputs = enhanced_model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor.unsqueeze(1))
        print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

Epoch [1/10], Loss: 1.2000, Val Loss: 1.1419
Epoch [2/10], Loss: 1.1419, Val Loss: 1.0839
Epoch [3/10], Loss: 1.0839, Val Loss: 1.0253
Epoch [4/10], Loss: 1.0252, Val Loss: 0.9654
Epoch [5/10], Loss: 0.9652, Val Loss: 0.9034
Epoch [6/10], Loss: 0.9031, Val Loss: 0.8383
Epoch [7/10], Loss: 0.8378, Val Loss: 0.7689
Epoch [8/10], Loss: 0.7683, Val Loss: 0.6941
Epoch [9/10], Loss: 0.6934, Val Loss: 0.6128
Epoch [10/10], Loss: 0.6119, Val Loss: 0.5240


In [ ]:
# Validation
# model.eval()
# with torch.no_grad():
#     val_outputs = model(X_val_tensor)
#     val_loss = criterion(val_outputs, y_val_tensor.unsqueeze(1))
#     print(f"Validation Loss: {val_loss.item():.4f}")

In [12]:
# # Testing
# with torch.no_grad():
#     test_outputs = model(X_test_tensor)
#     test_loss = criterion(test_outputs, y_test_tensor.unsqueeze(1))
#     print(f"Test Loss: {test_loss.item():.4f}")

# Evaluate on test set
enhanced_model.eval()
with torch.no_grad():
    test_outputs = enhanced_model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor.unsqueeze(1))
    print(f"Test Loss: {test_loss.item():.4f}")

Test Loss: 0.5222


In [ ]:
# import os

# # %cd "/content/drive/MyDrive/EE 199"
# print(os.getcwd())

In [13]:
# Assuming 'model' is your trained LSTM model
torch.save(enhanced_model.state_dict(), 'model.pt')

In [14]:
# Create an instance of your LSTM model
model = LSTMModel(input_size, hidden_size, output_size)

# Load the saved weights
model.load_state_dict(torch.load('model.pt'))
model.eval()  # Set the model to evaluation mode

LSTMModel(
  (lstm1): LSTM(3, 64, batch_first=True)
  (lstm2): LSTM(64, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
)

---------------------**Activate API Endpoint** before proceeding-------------------------

In [15]:
import requests


In [ ]:

# input_data = {
#     "input": [
#         [0.1, 0.2, 0.3],
#         [0.4, 0.5, 0.6],
#         [0.1, 0.2, 0.3],
#         [0.4, 0.5, 0.6],
#         [0.1, 0.2, 0.3],
#         [0.4, 0.5, 0.6],
#         [0.1, 0.2, 0.3],
#         [0.4, 0.5, 0.6],
#         [0.1, 0.2, 0.3],
#         [0.4, 0.5, 0.6],
        
#     ]
# }


In [ ]:

# response = requests.post('http://localhost:5000/predict', json=input_data).json()
# # print(response)

# threshold = 0.5
# RBP = response["output"]

# print("RBP =", RBP)
# if RBP >= threshold:
#     print("WARNING! Impending robbery.")


In [16]:
for i in range(int(n_samples*0.15)):
    input_data = {
    "input":  X_test_scaled.tolist()[i] 
}
    response = requests.post('http://localhost:5000/predict', json=input_data).json()

    threshold = 0.5
    RBP = response["output"]

    print(i, "RBP =", RBP, "| actual RBP =", y_test.tolist()[i])
    if RBP >= threshold:
        print("WARNING! Impending robbery.")

0 RBP = 0.2364712655544281 | actual RBP = 1
1 RBP = 0.25159475207328796 | actual RBP = 1
2 RBP = 0.24401068687438965 | actual RBP = 1
3 RBP = 0.2608254849910736 | actual RBP = 1
4 RBP = 0.22676408290863037 | actual RBP = 1
5 RBP = 0.25033605098724365 | actual RBP = 1
6 RBP = 0.22707965970039368 | actual RBP = 1
7 RBP = 0.25682753324508667 | actual RBP = 1
8 RBP = 0.2625955641269684 | actual RBP = 1
9 RBP = 0.2682943046092987 | actual RBP = 1
10 RBP = 0.2750760018825531 | actual RBP = 1
11 RBP = 0.23388680815696716 | actual RBP = 1
12 RBP = 0.2358507513999939 | actual RBP = 1
13 RBP = 0.2527351379394531 | actual RBP = 1
14 RBP = 0.24687108397483826 | actual RBP = 1


In [ ]:
# study first doen lstm
# work on dynamic input
# try solar dataset